In [1]:
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator
from yelp.errors import BusinessUnavailable
import os
import json
from pandas import DataFrame
import folium
import geojson
import random
import requests
import numpy as np
from geopy.distance import vincenty
import bokeh.plotting as plt

In [3]:
#Get Credentials
credentials = json.load(open('yelp_credentials.json'))

auth = Oauth1Authenticator(
    consumer_key=credentials['consumer_key'],
    consumer_secret=credentials['consumer_secret'],
    token=credentials['token'],
    token_secret=credentials['token_secret']
)

client = Client(auth)

In [53]:
#this is what we are after
#https://github.com/Yelp/yelp-python
bus_id='gregorys-coffee-new-york'
response= client.get_business(bus_id)
response.business.name

"Gregory's Coffee"

In [56]:
response = client.search('San Francisco')
response.businesses[0].name

'Tadu Ethiopian Kitchen'

In [ ]:
response = client.search(term='starbucks', location='07458')
response.businesses[1].name

In [19]:
def get_biz(name, area='New York', manual_override=0):
    area = area.lower().replace(' ', '-')
    name = name.lower().replace(' ', '-')
    bus_id = area + '-' + name + '-'
    i = 2
        # Run the first one through by hand.
    try:
        responses = [client.get_business(bus_id + '-')]
        # This can happen, and did, in the Starbucks case.
    except BusinessUnavailable:
        responses = []
        pass
    # The rest are handled by a loop.
    while True:
        bus_id = "{0}-{1}-{2}".format(name, area, i)
        try:
            response = client.get_business(bus_id)
        except BusinessUnavailable:
            # We manually check trouble spots.
            # But see the TODO.
            if requests.get('http://www.yelp.com/biz/' + bus_id).status_code != requests.codes.ok:
                break
            else:
                # Increment the counter but don't include the troubled ID.
                i += 1
                continue
        responses += [response]
        i += 1
    print("Ended `fetch_businesses()` on:", "http://www.yelp.com/biz/" + bus_id)
    return responses        

def frame(responses):
    """
    Given a list of yelp.obj.business_response.BusinessResponse objects like the one returns by fetch_businesses(),
    builds a coordinate-logging DataFrame out of them.
    """
    latitudes = [response.business.location.coordinate.latitude for response in responses]
    longitudes = [response.business.location.coordinate.longitude for response in responses]
    df = DataFrame({'latitude': latitudes, 'longitude': longitudes})
    df.index.name=responses[0].business.name
    return df


In [23]:
gregorys = get_biz('gregorys coffee')

Ended `fetch_businesses()` on: http://www.yelp.com/biz/gregorys-coffee-new-york-21


In [22]:
frame(gregorys)

,latitude,longitude
Gregory's Coffee,,
0,40.755527,-73.981826
1,40.754470,-73.988020
2,40.755447,-73.978218
3,40.747712,-73.989058
4,40.760942,-73.973070
5,40.747299,-73.989609
6,40.706075,-74.012867
7,40.705250,-74.006760
8,40.704325,-74.011700


In [42]:
folium.Map(location=[40.753889, -73.983611], zoom_start=13)
# jersey 41.07214,-74.1114984
# nyc 40.753889, -73.983611

In [30]:
type(frame(gregorys))

pandas.core.frame.DataFrame

In [38]:
def map_coordinates(df):
    """
    Returns a folium map of all of the coordinates stored in a coordinate DataFrame, like the one returned by frame().
    """
    map = folium.Map(location=[40.753889, -73.983611], zoom_start=5)
    for row in df.iterrows():
        map.simple_marker([row[1]['latitude'], row[1]['longitude']])
    return map

In [32]:
map_coordinates(frame(gregorys))

/Users/kathrynvasilaky/.pyenv/versions/3.5.1/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: simple_marker is deprecated. Use add_children(Marker) instead


In [35]:
starbucks = get_biz('starbucks coffee')
frame(starbucks)

Ended `fetch_businesses()` on: http://www.yelp.com/biz/starbucks-coffee-new-york-151


,latitude,longitude
Starbucks,,
0,40.760986,-73.970009
1,40.782574,-73.951447
2,40.714951,-74.006081
3,40.743800,-73.976830
4,40.804661,-73.966637
5,40.744259,-73.983749
6,40.756820,-73.981849
7,40.748929,-73.982806
8,40.756420,-73.975090


In [54]:
map_coordinates(frame(starbucks))

/Users/kathrynvasilaky/.pyenv/versions/3.5.1/lib/python3.5/site-packages/ipykernel/__main__.py:12: FutureWarning: simple_marker is deprecated. Use add_children(Marker) instead


In [43]:
area = 'new jersey'
name = 'starbucks coffee'
starbucks = get_biz('starbucks coffee')
frame(starbucks)

Ended `fetch_businesses()` on: http://www.yelp.com/biz/starbucks-coffee-new-york-151


,latitude,longitude
Starbucks,,
0,40.760986,-73.970009
1,40.782574,-73.951447
2,40.714951,-74.006081
3,40.743800,-73.976830
4,40.804661,-73.966637
5,40.744259,-73.983749
6,40.756820,-73.981849
7,40.748929,-73.982806
8,40.756420,-73.975090
